In [3]:
from gurobipy import Model, GRB, quicksum
import numpy as np

In [4]:
from itertools import product

num_weeks = 9
num_consmatches_1 = 4
num_consmatches_2 = 5

def valid_pattern(pattern):
    h_count = pattern.count('H')
    a_count = pattern.count('A')
    
    # Check the total H/A counts
    if not (
        (h_count == num_consmatches_1 and a_count == num_consmatches_2) or
        (h_count == num_consmatches_2 and a_count == num_consmatches_1)
    ):
        return False
    
    # Check for "breaks" only between odd/even round pairs
    for i in range(0, len(pattern), 2):
        if i + 1 < len(pattern):
            if pattern[i] == pattern[i + 1]:
                return False
    
    return True

all_patterns = product('HA', repeat=num_weeks)
valid_patterns = [
    ''.join(p) for p in all_patterns
    if valid_pattern(p)
]

num_patterns = len(valid_patterns)
print(f"Total valid patterns: {len(valid_patterns)}")
print(valid_patterns)


Total valid patterns: 32
['HAHAHAHAH', 'HAHAHAHAA', 'HAHAHAAHH', 'HAHAHAAHA', 'HAHAAHHAH', 'HAHAAHHAA', 'HAHAAHAHH', 'HAHAAHAHA', 'HAAHHAHAH', 'HAAHHAHAA', 'HAAHHAAHH', 'HAAHHAAHA', 'HAAHAHHAH', 'HAAHAHHAA', 'HAAHAHAHH', 'HAAHAHAHA', 'AHHAHAHAH', 'AHHAHAHAA', 'AHHAHAAHH', 'AHHAHAAHA', 'AHHAAHHAH', 'AHHAAHHAA', 'AHHAAHAHH', 'AHHAAHAHA', 'AHAHHAHAH', 'AHAHHAHAA', 'AHAHHAAHH', 'AHAHHAAHA', 'AHAHAHHAH', 'AHAHAHHAA', 'AHAHAHAHH', 'AHAHAHAHA']


In [5]:
solution_found = {'p': {(2, 4, 4): 1.0,
  (2, 6, 5): 1.0,
  (2, 13, 6): 1.0,
  (2, 16, 7): 1.0,
  (2, 25, 2): 1.0,
  (2, 27, 3): 1.0,
  (2, 29, 0): 1.0,
  (2, 31, 1): 1.0,
  (3, 4, 6): 1.0,
  (3, 6, 4): 1.0,
  (3, 13, 7): 1.0,
  (3, 16, 0): 1.0,
  (3, 25, 3): 1.0,
  (3, 27, 1): 1.0,
  (3, 29, 5): 1.0,
  (3, 31, 2): 1.0,
  (4, 6, 7): 1.0,
  (4, 13, 3): 1.0,
  (4, 16, 1): 1.0,
  (4, 25, 5): 1.0,
  (4, 27, 0): 1.0,
  (4, 29, 2): 1.0,
  (6, 13, 2): 1.0,
  (6, 16, 6): 1.0,
  (6, 25, 0): 1.0,
  (6, 29, 1): 1.0,
  (6, 31, 3): 1.0,
  (13, 16, 4): 1.0,
  (13, 25, 1): 1.0,
  (13, 27, 5): 1.0,
  (13, 31, 0): 1.0,
  (16, 27, 2): 1.0,
  (16, 29, 3): 1.0,
  (16, 31, 5): 1.0,
  (25, 27, 7): 1.0,
  (25, 29, 4): 1.0,
  (25, 31, 6): 1.0,
  (27, 29, 6): 1.0,
  (27, 31, 4): 1.0,
  (29, 31, 7): 1.0},
 'y': {2: 1.0,
  3: 1.0,
  4: 1.0,
  6: 1.0,
  13: 1.0,
  16: 1.0,
  25: 1.0,
  27: 1.0,
  29: 1.0,
  31: 1.0}}

In [6]:
selected_patterns = solution_found['y']
schedule = solution_found['p']

In [7]:
import pandas as pd

# Extract weeks 
weeks = sorted({t for _, _, t in schedule.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week
for t in weeks:
    week_pairs = []
    for (i, j, w) in schedule:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs


calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Pattern 2 vs Pattern 29,Pattern 2 vs Pattern 31,Pattern 2 vs Pattern 25,Pattern 2 vs Pattern 27,Pattern 2 vs Pattern 4,Pattern 2 vs Pattern 6,Pattern 2 vs Pattern 13,Pattern 2 vs Pattern 16
1,Pattern 3 vs Pattern 16,Pattern 3 vs Pattern 27,Pattern 3 vs Pattern 31,Pattern 3 vs Pattern 25,Pattern 3 vs Pattern 6,Pattern 3 vs Pattern 29,Pattern 3 vs Pattern 4,Pattern 3 vs Pattern 13
2,Pattern 4 vs Pattern 27,Pattern 4 vs Pattern 16,Pattern 4 vs Pattern 29,Pattern 4 vs Pattern 13,Pattern 13 vs Pattern 16,Pattern 4 vs Pattern 25,Pattern 6 vs Pattern 16,Pattern 4 vs Pattern 6
3,Pattern 6 vs Pattern 25,Pattern 6 vs Pattern 29,Pattern 6 vs Pattern 13,Pattern 6 vs Pattern 31,Pattern 25 vs Pattern 29,Pattern 13 vs Pattern 27,Pattern 25 vs Pattern 31,Pattern 25 vs Pattern 27
4,Pattern 13 vs Pattern 31,Pattern 13 vs Pattern 25,Pattern 16 vs Pattern 27,Pattern 16 vs Pattern 29,Pattern 27 vs Pattern 31,Pattern 16 vs Pattern 31,Pattern 27 vs Pattern 29,Pattern 29 vs Pattern 31


In [8]:
# Create the list of teams and ranking
teams = [
    "Brazil", "Argentina", "Uruguay", 
    "Ecuador", "Peru", "Colombia", 
    "Chile", "Paraguay", "Bolivia", "Venezuela"
]

ranking = {
    1: teams[:2],  
    2: teams[2:6],  
    3: teams[6:]    
}

# Assign a unique number to each team
team_numbers = {i : team for i, team in enumerate(teams)}

# Create a reverse mapping to look up the number of a team
team_to_number = {team: number for number, team in team_numbers.items()}


stronger_teams = {}

# Iterate through each team and determine stronger teams
for position, teams_at_position in ranking.items():
    for team in teams_at_position:
        team_number = team_to_number[team]  
        stronger_teams[team_number] = [
            team_to_number[stronger_team]
            for higher_position in range(1, position)  
            for stronger_team in ranking[higher_position]
        ]


print("Team Numbers:", team_numbers)
print("Stronger Teams:", stronger_teams)

Team Numbers: {0: 'Brazil', 1: 'Argentina', 2: 'Uruguay', 3: 'Ecuador', 4: 'Peru', 5: 'Colombia', 6: 'Chile', 7: 'Paraguay', 8: 'Bolivia', 9: 'Venezuela'}
Stronger Teams: {0: [], 1: [], 2: [0, 1], 3: [0, 1], 4: [0, 1], 5: [0, 1], 6: [0, 1, 2, 3, 4, 5], 7: [0, 1, 2, 3, 4, 5], 8: [0, 1, 2, 3, 4, 5], 9: [0, 1, 2, 3, 4, 5]}


In [9]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
m = Model("Team_to_Pattern_Allocation")

# Parameters
teams = range(10)  
patterns = selected_patterns 
weeks = range(9)  


# Given calendar
calendar = schedule

# Decision variables
z = m.addVars(teams, patterns, vtype=GRB.BINARY, name="z")  # Team-to-pattern allocation
s = m.addVars(teams, range(8), vtype=GRB.BINARY, name="s")  # Carry-over effect
x = m.addVars(teams, teams, weeks, vtype=GRB.BINARY, name="x")


# Objective: Minimize carry-over effect
m.setObjective(quicksum(s[i, k] for i in teams for k in range(8)), GRB.MINIMIZE)


# Constraint C2: Each team must be assigned to exactly one pattern
for i in teams:
    m.addConstr(quicksum(z[i, p] for p in patterns) == 1, name=f"team_pattern_allocation_{i}")

# Each pattern must be assigned to exactly one team
for p in patterns:
    m.addConstr(quicksum(z[i,p] for i in teams) == 1, name=f"team_pattern_unique_allocation_{p}")

# Constraint: Relate calendar pattern pairings to matches between teams


for (p_i, p_j, t) in calendar.keys():
    for i in teams:
        for j in teams:
            if i < j:
                m.addConstr(x[i, j, t] >= z[i, p_i] + z[i, p_j] + z[j, p_j] + z[j, p_i] - 1, name=f"match_constraint_link_{i}_{j}_{t}")

for i in teams:
    for j in teams:
        if i < j:
            m.addConstr(quicksum(x[i,j,t] for t in weeks) == 1)

for t in weeks:
    for i in teams:
            m.addConstr(quicksum(x[i,j,t] for j in teams if i<j) + quicksum(x[j,i,t] for j in teams if j<i) == 1)


for i in teams:
     for j in teams:
          if i>=j:
               for t in weeks:
                    x[i,j,t].ub = 0

# Constraint: Carry-over effect between weeks
for i in teams:
    for k in range(8):
        m.addConstr(
            quicksum(x[i, j, k] for j in stronger_teams[i] if i<j) + 
            quicksum(x[j, i, k] for j in stronger_teams[i] if j<i) +
            quicksum(x[i, j, k+1] for j in stronger_teams[i] if i<j) + 
            quicksum(x[j, i, k+1] for j in stronger_teams[i] if j<i) 
            <= 1 + s[i, k],
            name=f"carry_over_effect_{i}_{k}"
        )


        m.addConstr(
            quicksum(x[i, j, k] for j in stronger_teams[i] if i<j) + 
            quicksum(x[j, i, k] for j in stronger_teams[i] if j<i)
            >= s[i, k]
        )
        m.addConstr(
            quicksum(x[i, j, k+1] for j in stronger_teams[i] if i<j) + 
            quicksum(x[j, i, k+1] for j in stronger_teams[i] if j<i)
            >= s[i, k]
        )


m.optimize()



if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    allocation = {(i, p): z[i, p].x for i in teams for p in patterns if z[i, p].x > 0.5}
    carry_over = {(i, k): s[i, k].x for i in teams for k in range(8) if s[i, k].x > 0.5}
    matches_opt = {(i, j, t): x[i, j, t].x for i in teams for j in teams for t in weeks if x[i, j, t].x > 0.5}
    print("Team-to-pattern allocation:", allocation)
    print("Carry-over effect:", carry_over)
    print("Matches between teams:", matches_opt)
else:
    print("No optimal solution found")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-04
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2195 rows, 1080 columns and 11679 nonzeros
Model fingerprint: 0x299cb8a5
Variable types: 0 continuous, 1080 integer (1080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 48 rows and 511 columns
Presolve time: 0.01s
Presolved: 2147 rows, 569 columns, 11631 nonzeros
Variable types: 0 continuous, 569 integer (569 binary)
Found heuristic solution: objective 13.0000000

Root relaxation: objective 8.000000e+00, 717 iterations, 0.05 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objectiv

In [10]:
pattern_to_team = {team_numbers[team] : p for (team, p), value in allocation.items() if value > 0.5}
weeks = {}

for (team1, team2, week), value in matches_opt.items():
    if value == 1.0:  
        if week not in weeks:
            weeks[week] = []
        key = pattern_to_team[team_numbers[team1]]  
        if valid_patterns[key][week] == 'H':
            match = f"{team_numbers[team1]} vs {team_numbers[team2]}"
        else:
            match = f"{team_numbers[team2]} vs {team_numbers[team1]}"
        if match not in weeks[week]:
            weeks[week].append(match)


sorted_weeks_example = sorted(weeks.keys())

max_matches_example = max(len(matches) for matches in weeks.values())

formatted_data_example = {
    f"Week {week}": weeks[week] + [None] * (max_matches_example - len(weeks[week]))
    for week in sorted_weeks_example
}

formatted_df_example = pd.DataFrame(formatted_data_example)
formatted_df_example

,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Brazil vs Bolivia,Argentina vs Brazil,Brazil vs Paraguay,Ecuador vs Brazil,Venezuela vs Brazil,Brazil vs Peru,Colombia vs Brazil,Brazil vs Uruguay,Brazil vs Chile
1,Peru vs Argentina,Colombia vs Uruguay,Uruguay vs Argentina,Argentina vs Colombia,Bolivia vs Argentina,Argentina vs Venezuela,Argentina vs Chile,Ecuador vs Argentina,Paraguay vs Argentina
2,Uruguay vs Chile,Ecuador vs Peru,Venezuela vs Ecuador,Paraguay vs Uruguay,Peru vs Uruguay,Uruguay vs Bolivia,Uruguay vs Venezuela,Peru vs Colombia,Uruguay vs Ecuador
3,Paraguay vs Ecuador,Chile vs Venezuela,Peru vs Bolivia,Chile vs Peru,Chile vs Ecuador,Ecuador vs Colombia,Bolivia vs Ecuador,Chile vs Bolivia,Peru vs Venezuela
4,Venezuela vs Colombia,Bolivia vs Paraguay,Colombia vs Chile,Bolivia vs Venezuela,Colombia vs Paraguay,Paraguay vs Chile,Paraguay vs Peru,Venezuela vs Paraguay,Colombia vs Bolivia
